<a href="https://colab.research.google.com/github/edumdp-dev/swift/blob/main/V2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install pandas unidecode spacy nltk scikit-learn seaborn matplotlib torch transformers scipy tqdm openpyxl pysentimiento

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 24.3 MB/s eta 0:00:00


In [ ]:
# -------------------------------------------------------------------
# BLOCO ÚNICO COMPLETO: (VERSÃO COM LABELS DE TÓPICO OTIMIZADAS)
# Carrega dados, limpa, classifica, analisa TF-IDF e exporta tudo.
# -------------------------------------------------------------------

# --- 1. Imports Gerais ---
import pandas as pd
import re
import string
import unidecode
import spacy
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from scipy.special import softmax
import time
from tqdm.auto import tqdm # Para barras de progresso
import os # Adicionar esta linha

# Configuração do Seaborn para gráficos
sns.set_theme(style="whitegrid")

# --- 2. Download de Recursos NLTK (se necessário) ---
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Baixando lista de stopwords do NLTK...")
    nltk.download('stopwords')

# --- 3. Carregamento dos Dados (Baseado na Célula 10) ---
print("Carregando dados do Excel...")
try:
    df = pd.read_excel('NPS-Comentarios-2024-Louveira_VilaAndrade.xlsx')
    print("Arquivo 'NPS-Comentarios-2024-Louveira_VilaAndrade.xlsx' lido com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivo 'NPS-Comentarios-2024-Louveira_VilaAndrade.xlsx' não encontrado.")
    # Cria um DF vazio para evitar que o resto do script quebre
    df = pd.DataFrame(columns=['Ano', 'Centro', 'Mes', 'Semana', 'Data Avaliação', 'Comentario', 'classificacao', 'NPS'])
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo Excel: {e}")
    df = pd.DataFrame(columns=['Ano', 'Centro', 'Mes', 'Semana', 'Data Avaliação', 'Comentario', 'classificacao', 'NPS'])

# --- 4. Criação dos Sub-DataFrames (Baseado na Célula 12) ---
print("Criando sub-dataframes por classificação (detrator, neutro, promotor)...")
df_detratores = df[df['classificacao'] == 'detrator'].copy()
df_neutros = df[df['classificacao'] == 'neutro'].copy()
df_promotores = df[df['classificacao'] == 'promotor'].copy()

# --- 5. Funções de Limpeza e Lematização (Baseado na Célula 13) ---
print("Definindo funções de limpeza de texto...")

# Compilação de Regex (para eficiência)
EMOJI_PATTERN = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)
NOISE_PATTERN = re.compile(r'http\S+|www\S+|https\S+|@\S+|#\S+|\d+')

# Stopwords customizadas (combinando Célula 13 e Célula 21)
portuguese_stops_base = stopwords.words('portuguese')
CUSTOM_STOPWORDS = portuguese_stops_base + [
    'pra', 'pro', 'vcs', 'vc', 'tá',
    "nps", "pesquisa", "atendimento", "empresa", "servico", "produto",
    "recomendar", "recomendaria", "nota", "experience", "pode", "ser",
    "dever", "mim", "sobre", "talvez", "apenas", "coisa"
]

def load_spacy_model(model_name='pt_core_news_sm'):
    """
    Carrega o modelo spaCy e adiciona stopwords customizadas.
    """
    try:
        nlp = spacy.load(model_name)
    except IOError:
        print(f"Modelo {model_name} não encontrado. Baixando...")
        spacy.cli.download(model_name)
        nlp = spacy.load(model_name)

    # Adiciona a lista customizada ao pipeline do spaCy
    for word in CUSTOM_STOPWORDS:
        nlp.Defaults.stop_words.add(word)
        nlp.Defaults.stop_words.add(unidecode.unidecode(word.lower()))

    print(f"Modelo spaCy '{model_name}' carregado com {len(nlp.Defaults.stop_words)} stopwords.")
    return nlp

def process_text_pipeline(text: str, nlp_model):
    """
    Limpa e lematiza um único texto usando o pipeline spaCy otimizado.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    text = EMOJI_PATTERN.sub(r'', text)
    text = NOISE_PATTERN.sub(r'', text)
    doc = nlp_model(text)
    lemmatized_tokens = []

    for token in doc:
        if (
            not token.is_stop
            and not token.is_punct
            and not token.is_space
            and token.pos_ != 'SYM'
            and len(token.lemma_) > 2
        ):
            lemma = token.lemma_
            lemma = lemma.lower()
            lemma = unidecode.unidecode(lemma)
            lemmatized_tokens.append(lemma)

    return " ".join(lemmatized_tokens)

# --- 6. Execução da Limpeza (Baseado na Célula 14) ---
# (Esta etapa cria a coluna 'comentario_limpo', corrigindo o erro)
print("\nIniciando limpeza e lematização dos textos...")
try:
    nlp = load_spacy_model('pt_core_news_sm')

    tqdm.pandas(desc="Limpando DF Principal")
    df['comentario_limpo'] = df['Comentario'].progress_apply(lambda text: process_text_pipeline(text, nlp))

    tqdm.pandas(desc="Limpando Detratores")
    df_detratores['comentario_limpo'] = df_detratores['Comentario'].progress_apply(lambda text: process_text_pipeline(text, nlp))

    tqdm.pandas(desc="Limpando Neutros")
    df_neutros['comentario_limpo'] = df_neutros['Comentario'].progress_apply(lambda text: process_text_pipeline(text, nlp))

    tqdm.pandas(desc="Limpando Promotores")
    df_promotores['comentario_limpo'] = df_promotores['Comentario'].progress_apply(lambda text: process_text_pipeline(text, nlp))

    print("✅ Limpeza de texto concluída.")
except Exception as e:
    print(f"ERRO durante a limpeza de texto: {e}")
    # Adiciona coluna vazia para evitar que o script pare
    if 'comentario_limpo' not in df.columns:
        df['comentario_limpo'] = ""
        df_detratores['comentario_limpo'] = ""
        df_neutros['comentario_limpo'] = ""
        df_promotores['comentario_limpo'] = ""


# --- 7. Classificação de Sentimento e Tópico (Baseado na Célula 17) ---
# (Necessário para a segunda exportação)
print("\nIniciando classificação de Sentimento e Tópico (IA)...")

# Tentar usar GPU
device = 0 if torch.cuda.is_available() else -1
if device == 0:
    print("✅ GPU (CUDA) detectada. Processamento rápido.")
else:
    print("⚠️ GPU não detectada. Processando em CPU (lento).")

# Modelos
MODELO_SENTIMENTO = "pysentimiento/bertweet-pt-sentiment"
MODELO_TOPICO = "joeddav/xlm-roberta-large-xnli"

try:
    pipe_sentimento = pipeline("text-classification", model=MODELO_SENTIMENTO, device=device)
    pipe_topico = pipeline("zero-shot-classification", model=MODELO_TOPICO, device=device)

    # Labels Novas (Mais descritivas para melhorar a acurácia):
    print("Usando LABELS DE TÓPICO OTIMIZADAS.")
    LABELS_TOPICO = [
        "Atendimento e cordialidade da equipe",
        "Qualidade do produto",
        "Disponibilidade e variedade de produtos (Estoque)",
        "Preço e promoções",
        "Ambiente e organização da loja",
        "Entrega ao cliente (Prazo, frete)", # Específico para o cliente final
        "Experiência no App ou Site (Digital)",
        "Logística e Abastecimento (Processo Interno)", # Nova label para capturar o outro "tipo" de entrega
        "Comentário Genérico / Elogio Vazio",
        "Outros"
    ]
    # ==================================================================


    # Prepara o DataFrame para processamento (usando 'df' que já tem 'comentario_limpo')
    df_proc = df.copy()

    LISTA_GENERICA_PREFIXOS = [
        'nada a declarar', 'tudo certo', 'ok', 'tudo ok', 'obrigado',
        'obrigada', 'tá ótimo', 'ta otimo', 'nada', 'nenhum',
        'sem comentarios', 'sem comentário'
    ]

    def is_valid_comment(comment):
        return isinstance(comment, str) and comment.strip() != '-' and len(comment.strip()) > 1

    def is_generic_comment(comment):
        if not isinstance(comment, str): return False
        comment_norm = comment.strip().lower()
        for prefixo in LISTA_GENERICA_PREFIXOS:
            if comment_norm.startswith(prefixo):
                return True
        return False

    df_proc['is_valid'] = df_proc['Comentario'].apply(is_valid_comment)
    df_proc['is_generic'] = df_proc['Comentario'].apply(is_generic_comment)

    comentarios_para_sentimento = df_proc.loc[df_proc['is_valid'], 'Comentario'].tolist()
    mask_topico_ia = (df_proc['is_valid']) & (~df_proc['is_generic'])
    comentarios_para_topico_ia = df_proc.loc[mask_topico_ia, 'Comentario'].tolist()

    df_proc['sentimento_modelo'] = "Sem Comentário"
    df_proc['topico_modelo'] = "Nenhum"

    # Processamento Sentimento
    if comentarios_para_sentimento:
        print("Processando Sentimentos...")
        start_time_sent = time.time()
        resultados_sent_raw = pipe_sentimento(comentarios_para_sentimento, batch_size=16, truncation=True)
        label_map_sent = {'POS': 'Positivo', 'NEG': 'Negativo', 'NEU': 'Neutro'}
        resultados_sentimento_lista = [label_map_sent.get(r['label'], r['label']) for r in resultados_sent_raw]
        df_proc.loc[df_proc['is_valid'], 'sentimento_modelo'] = resultados_sentimento_lista
        print(f"✅ Sentimentos concluídos em {time.time() - start_time_sent:.2f} seg.")

    # Processamento Tópico
    if comentarios_para_topico_ia:
        print("Processando Tópicos (IA)...")
        start_time_top = time.time()
        resultados_top_raw = pipe_topico(comentarios_para_topico_ia, candidate_labels=LABELS_TOPICO, multi_label=False, batch_size=16, truncation=True)
        resultados_topico_lista = [r['labels'][0] for r in resultados_top_raw]
        df_proc.loc[mask_topico_ia, 'topico_modelo'] = resultados_topico_lista
        print(f"✅ Tópicos (IA) concluídos em {time.time() - start_time_top:.2f} seg.")

    # Regras Finais de Tópico
    print("Aplicando regras finais para tópicos...")
    df_proc.loc[df_proc['is_generic'], 'topico_modelo'] = "Comentário Genérico / Elogio Vazio"
    df_proc.loc[~df_proc['is_valid'], 'topico_modelo'] = "Nenhum"
    print("✅ Classificação de Sentimento/Tópico concluída.")

except Exception as e:
    print(f"ERRO durante a Classificação IA (Célula 17): {e}")
    # Cria df_proc vazio para não quebrar a exportação 2
    df_proc = pd.DataFrame(columns=['Centro', 'sentimento_modelo', 'classificacao', 'NPS', 'topico_modelo'])


# --- 8. Funções de Análise TF-IDF (Score + Frequência) (Baseado no Bloco 1) ---
print("\nDefinindo funções de análise TF-IDF (Score + Frequência)...")

def analyze_and_get_top_ngrams(text_data, n_top, ngram_range, min_df, stop_words_list):
    # Definindo um valor mínimo de documentos (min_df)
    min_df_threshold = max(min_df, 1) # Garante que seja pelo menos 1

    results_df = pd.DataFrame(columns=['term', 'score', 'frequency'])
    if text_data.empty or len(text_data) < min_df_threshold:
        # print(f"P_ulando: poucos dados ({len(text_data)} docs, min_df={min_df_threshold})")
        return results_df

    try:
        tfidf_vectorizer = TfidfVectorizer(
            ngram_range=ngram_range, min_df=min_df_threshold, stop_words=stop_words_list,
            max_df=0.90, sublinear_tf=True
        )
        tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)
        feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

        if not feature_names_tfidf.any():
            # print("Nenhum feature_name encontrado com min_df, tentando min_df=1")
            # Tenta relaxar a restrição se falhou
            min_df_threshold = 1
            if len(text_data) < min_df_threshold:
                 return results_df

            tfidf_vectorizer = TfidfVectorizer(
                ngram_range=ngram_range, min_df=min_df_threshold, stop_words=stop_words_list,
                max_df=0.90, sublinear_tf=True
            )
            tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)
            feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

        if not feature_names_tfidf.any():
            # print("Nenhum feature_name encontrado mesmo com min_df=1. Pulando.")
            return results_df

        mean_tfidf = np.array(tfidf_matrix.mean(axis=0)).flatten()
        tfidf_df = pd.DataFrame({'term': feature_names_tfidf, 'score': mean_tfidf})

        count_vectorizer = CountVectorizer(
            ngram_range=ngram_range, min_df=min_df_threshold, stop_words=stop_words_list,
            max_df=0.90, vocabulary=feature_names_tfidf
        )
        count_matrix = count_vectorizer.fit_transform(text_data)
        total_counts = np.array(count_matrix.sum(axis=0)).flatten()
        count_df = pd.DataFrame({'term': feature_names_tfidf, 'frequency': total_counts})

        results_df = pd.merge(tfidf_df, count_df, on='term', how='left')
        results_df['frequency'] = results_df['frequency'].fillna(0).astype(int)

        top_terms_df = results_df.sort_values(by='score', ascending=False).head(n_top)
        return top_terms_df

    except ValueError as e:
        # Isso acontece se o vocabulário estiver vazio (ex: todos são stopwords)
        # print(f"Erro no TF-IDF (provavelmente vocabulário vazio): {e}")
        return pd.DataFrame(columns=['term', 'score', 'frequency'])

def plot_top_ngrams_with_freq(df, title, palette, xlabel='Importância (Média TF-IDF)'):
    if df.empty:
        print(f"Não há dados para plotar o gráfico: {title}")
        return

    df_plot = df.sort_values(by='score', ascending=False)

    # Criar diretório de gráficos se não existir
    output_dir = "graficos_tfidf_centro"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    plt.figure(figsize=(12, 7))
    barplot = sns.barplot(
        x='score',
        y='term',
        data=df_plot,
        palette=palette
    )
    plt.title(title, fontsize=16, pad=15)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Bigrama', fontsize=12)

    barplot.bar_label(barplot.containers[0], fmt='%.3f', padding=5)

    freqs = df_plot['frequency'].values
    max_score = df_plot['score'].max()

    for i, (patch, freq) in enumerate(zip(barplot.patches, freqs)):
        barplot.text(patch.get_width() + (max_score * 0.01),
                     patch.get_y() + patch.get_height() / 2,
                     f'(Freq: {freq})',
                     va='center', ha='left', fontsize=9, color='#555555')

    plt.tight_layout()

    filename = os.path.join(output_dir, f"TFIDF_{title.replace(' ', '_').replace('/', '').replace('(', '').replace(')', '')}.png")
    plt.savefig(filename)
    print(f"Gráfico salvo como: {filename}")
    # plt.show() # Removido plt.show() para rodar de forma limpa no backend
    plt.close() # Fecha a figura para liberar memória


# --- 9. Execução Análise TF-IDF por Centro ---
print("\n--- INICIANDO ANÁLISE TF-IDF (SCORE + FREQUÊNCIA) POR CENTRO E CLASSIFICAÇÃO ---")

# Parâmetros do TF-IDF (definidos aqui para referência)
n_top = 15
ngram_range = (2, 2) # Apenas bigramas
min_df_threshold = 2 # Ignora termos que aparecem em menos de 2 comentários

all_tfidf_results = []
# Usamos o 'df' que agora TEM 'comentario_limpo'
centros_unicos = df['Centro'].unique()
classificacoes_unicas = ['promotor', 'neutro', 'detrator']

for centro in centros_unicos:
    for classificacao in classificacoes_unicas:

        subset_df = df[(df['Centro'] == centro) & (df['classificacao'] == classificacao)]
        text_data = subset_df['comentario_limpo'].dropna().loc[subset_df['comentario_limpo'].str.len() > 0]

        if not text_data.empty:
            top_ngrams_df = analyze_and_get_top_ngrams(
                text_data,
                n_top=n_top,
                ngram_range=ngram_range,
                min_df=min_df_threshold,
                stop_words_list=CUSTOM_STOPWORDS # Usando a lista de stopwords completa
            )

            if not top_ngrams_df.empty:
                top_ngrams_df['Centro'] = centro
                top_ngrams_df['Classificacao'] = classificacao
                all_tfidf_results.append(top_ngrams_df)

if all_tfidf_results:
    df_tfidf_completo = pd.concat(all_tfidf_results, ignore_index=True)

    print("\n--- Análise TF-IDF (Score + Frequência) Concluída ---")
    print("Amostra dos resultados (DataFrame 'df_tfidf_completo'):")
    display(df_tfidf_completo.head())

    print(f"\n--- Gerando Gráficos (com Frequência) ---")
    for centro in centros_unicos:
        for classificacao, palette in [('detrator', 'viridis_r'), ('promotor', 'viridis'), ('neutro', 'coolwarm')]:
            data_to_plot = df_tfidf_completo[
                (df_tfidf_completo['Centro'] == centro) &
                (df_tfidf_completo['Classificacao'] == classificacao)
            ]
            if not data_to_plot.empty:
                plot_title = f'Top Bigramas - {classificacao.capitalize()}s ({centro})'
                plot_top_ngrams_with_freq(data_to_plot, plot_title, palette)
            else:
                print(f"Sem dados de bigrama para plotar: {classificacao.capitalize()}s ({centro})")

    # --- EXPORTAÇÃO 1 (TF-IDF + Frequência) ---
    print("\n--- INICIANDO EXPORTAÇÃO 1 (TF-IDF + FREQUÊNCIA) ---")
    df_para_exportar_tfidf = df_tfidf_completo.rename(columns={
        'Centro': 'Loja/Centro',
        'term': 'Bigrama',
        'Classificacao': 'Classificacao',
        'score': 'Score_TFIDF',
        'frequency': 'Frequencia'
    })

    colunas_exportar_tfidf = ['Loja/Centro', 'Bigrama', 'Classificacao', 'Score_TFIDF', 'Frequencia']
    df_para_exportar_tfidf = df_para_exportar_tfidf[colunas_exportar_tfidf]

    output_filename_tfidf = "NPS_TFIDF_Bigramas_COM_FREQ.csv"
    df_para_exportar_tfidf.to_csv(output_filename_tfidf, index=False, encoding='utf-8-sig')

    print(f"Resultados da análise TF-IDF (com Frequência) exportados com sucesso para:")
    print(f"-> {output_filename_tfidf}")

else:
    print("\nNenhum resultado de TF-IDF foi gerado. Verifique os dados de entrada.")
    # Cria df_tfidf_completo vazio para não quebrar a exportação 1
    df_tfidf_completo = pd.DataFrame(columns=['Centro', 'term', 'Classificacao', 'score', 'frequency'])


# --- EXPORTAÇÃO 2 (Classificação/Sentimento) ---
# (Modificado para incluir a nova coluna 'topico_modelo')
print("\n--- INICIANDO EXPORTAÇÃO 2 (CLASSIFICAÇÃO/SENTIMENTO/TÓPICO) ---")
try:
    df_export_sentimento = df_proc.rename(columns={
        'Centro': 'Loja/Centro',
        'sentimento_modelo': 'Sentimento',
        'topico_modelo': 'Topico', # Adicionando a nova coluna
        'classificacao': 'Classificacao',
        'NPS': 'Score',
        'Comentario': 'Comentario_Original',
        'comentario_limpo': 'Comentario_Limpo'
    })

    colunas_exportar_alt = [
        'Loja/Centro',
        'Data Avaliação', # Adicionando data para contexto
        'Score',
        'Classificacao',
        'Sentimento',
        'Topico',
        'Comentario_Original',
        'Comentario_Limpo'
        ]

    # Pega apenas as colunas que realmente existem no df
    colunas_existentes = [col for col in colunas_exportar_alt if col in df_export_sentimento.columns]

    # Adiciona colunas que podem não estar na lista original mas existem em df_proc
    if 'Data Avaliação' not in colunas_existentes and 'Data Avaliação' in df_proc.columns:
        colunas_existentes.insert(1, 'Data Avaliação')

    df_export_sentimento = df_export_sentimento[colunas_existentes]

    output_filename_alt = "NPS_Classificacao_Completa.csv"
    df_export_sentimento.to_csv(output_filename_alt, index=False, encoding='utf-8-sig')

    print(f"Resultados da classificação completa (Sentimento/Tópico) exportados com sucesso para:")
    print(f"-> {output_filename_alt}")

except Exception as e:
    print(f"\nOcorreu um erro durante a exportação da classificação de sentimento: {e}")

# ==================================================================
# === INÍCIO DO NOVO BLOCO (SEÇÃO 10 e 11) ===
# ==================================================================

# --- 10. Execução Análise TF-IDF por Tópico e Sentimento (NOVO) ---
print("\n--- INICIANDO ANÁLISE TF-IDF (SCORE + FREQUÊNCIA) POR TÓPICO E SENTIMENTO ---")

all_tfidf_results_topico = []
# Usamos o 'df_proc' que tem as colunas 'sentimento_modelo' e 'topico_modelo'
centros_unicos_proc = df_proc['Centro'].unique()
sentimentos_unicos = df_proc['sentimento_modelo'].unique()
topicos_unicos = df_proc['topico_modelo'].unique()

# Parâmetros (reutilizando os da Seção 9)
# n_top = 15
# ngram_range = (2, 2)
# min_df_threshold = 2

for centro in tqdm(centros_unicos_proc, desc="Analisando Centros"):
    for sentimento in sentimentos_unicos:
        for topico in topicos_unicos:

            # Pular categorias não informativas para TF-IDF
            if topico in ["Nenhum", "Comentário Genérico / Elogio Vazio", "Outros"] or sentimento == "Sem Comentário":
                continue

            # Filtra o dataframe 'df_proc'
            subset_df = df_proc[
                (df_proc['Centro'] == centro) &
                (df_proc['sentimento_modelo'] == sentimento) &
                (df_proc['topico_modelo'] == topico)
            ]

            text_data = subset_df['comentario_limpo'].dropna().loc[subset_df['comentario_limpo'].str.len() > 0]

            # Garantir que temos dados suficientes (usando o mesmo min_df_threshold)
            if not text_data.empty and len(text_data) >= min_df_threshold:
                top_ngrams_df = analyze_and_get_top_ngrams(
                    text_data,
                    n_top=n_top,
                    ngram_range=ngram_range,
                    min_df=min_df_threshold,
                    stop_words_list=CUSTOM_STOPWORDS
                )

                if not top_ngrams_df.empty:
                    top_ngrams_df['Centro'] = centro
                    top_ngrams_df['Sentimento'] = sentimento
                    top_ngrams_df['Categoria'] = topico # Nome solicitado pelo usuário
                    all_tfidf_results_topico.append(top_ngrams_df)

if all_tfidf_results_topico:
    df_tfidf_topico_completo = pd.concat(all_tfidf_results_topico, ignore_index=True)

    print("\n--- Análise TF-IDF por Tópico/Sentimento Concluída ---")
    print("Amostra dos resultados (DataFrame 'df_tfidf_topico_completo'):")
    display(df_tfidf_topico_completo.head())

    # --- 11. EXPORTAÇÃO 3 (TF-IDF por Tópico/Sentimento) (NOVO) ---
    print("\n--- INICIANDO EXPORTAÇÃO 3 (TF-IDF POR TÓPICO/SENTIMENTO) ---")

    # Renomear colunas para o formato exato solicitado
    df_para_exportar_topico = df_tfidf_topico_completo.rename(columns={
        'term': 'Bigrama',
        'score': 'Score_TFIDF',
        'frequency': 'Frequencia'
        # 'Centro', 'Sentimento', 'Categoria' já estão corretos
    })

    # Ordenar colunas conforme solicitado
    colunas_exportar_topico = ['Centro', 'Bigrama', 'Sentimento', 'Categoria', 'Score_TFIDF', 'Frequencia']
    df_para_exportar_topico = df_para_exportar_topico[colunas_exportar_topico]

    output_filename_topico = "NPS_TFIDF_Bigramas_POR_TOPICO_SENTIMENTO.csv"
    df_para_exportar_topico.to_csv(output_filename_topico, index=False, encoding='utf-8-sig')

    print(f"Resultados da análise TF-IDF por Tópico/Sentimento exportados com sucesso para:")
    print(f"-> {output_filename_topico}")

else:
    print("\nNenhum resultado de TF-IDF por Tópico/Sentimento foi gerado (verifique se há dados suficientes).")

# ==================================================================
# === FIM DO NOVO BLOCO ===
# ==================================================================

print("\n--- SCRIPT COMPLETO CONCLUÍDO ---")

Carregando dados do Excel...
Arquivo 'NPS-Comentarios-2024-Louveira_VilaAndrade.xlsx' lido com sucesso.
Criando sub-dataframes por classificação (detrator, neutro, promotor)...
Definindo funções de limpeza de texto...

Iniciando limpeza e lematização dos textos...
Modelo spaCy 'pt_core_news_sm' carregado com 548 stopwords.


Limpando DF Principal:   0%|          | 0/1085 [00:00<?, ?it/s]

Limpando Detratores:   0%|          | 0/88 [00:00<?, ?it/s]

Limpando Neutros:   0%|          | 0/131 [00:00<?, ?it/s]

Limpando Promotores:   0%|          | 0/866 [00:00<?, ?it/s]

...

Gráfico salvo como: graficos_tfidf_centro/TFIDF_Top_Bigramas_-_Neutros_L5235-LOUVEIRA.png

--- INICIANDO EXPORTAÇÃO 1 (TF-IDF + FREQUÊNCIA) ---
Resultados da análise TF-IDF (com Frequência) exportados com sucesso para:
-> NPS_TFIDF_Bigramas_COM_FREQ.csv

--- INICIANDO EXPORTAÇÃO 2 (CLASSIFICAÇÃO/SENTIMENTO/TÓPICO) ---
Resultados da classificação completa (Sentimento/Tópico) exportados com sucesso para:
-> NPS_Classificacao_Completa.csv

--- INICIANDO ANÁLISE TF-IDF (SCORE + FREQUÊNCIA) POR TÓPICO E SENTIMENTO ---


Analisando Centros:   0%|          | 0/2 [00:00<?, ?it/s]


--- Análise TF-IDF por Tópico/Sentimento Concluída ---
Amostra dos resultados (DataFrame 'df_tfidf_topico_completo'):


,term,score,frequency,Centro,Sentimento,Categoria
0,faltar item,0.250000,3,L5082-VILA ANDRADE (1346),Neutro,Experiência no App ou Site (Digital)
1,preco acessivel,0.068966,2,L5082-VILA ANDRADE (1346),Neutro,Preço e promoções
2,entregar dia,0.285714,2,L5082-VILA ANDRADE (1346),Neutro,"Entrega ao cliente (Prazo, frete)"
3,melhorar preco,0.300000,3,L5082-VILA ANDRADE (1346),Positivo,Preço e promoções
4,entregar dia,0.204883,3,L5082-VILA ANDRADE (1346),Negativo,"Entrega ao cliente (Prazo, frete)"



--- INICIANDO EXPORTAÇÃO 3 (TF-IDF POR TÓPICO/SENTIMENTO) ---
Resultados da análise TF-IDF por Tópico/Sentimento exportados com sucesso para:
-> NPS_TFIDF_Bigramas_POR_TOPICO_SENTIMENTO.csv

--- SCRIPT COMPLETO CONCLUÍDO ---


In [ ]:
# CÉLULA DE AUTENTICAÇÃO E INSTALAÇÃO (Rode 1x no topo)

!pip install --upgrade gspread gspread-dataframe

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# ----- ESTA É A LINHA CORRIGIDA -----
from gspread_dataframe import set_with_dataframe

try:
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Autenticação com Google Colab e Google Sheets concluída!")
except Exception as e:
    print(f"⚠️ Erro na autenticação: {e}")

✅ Autenticação com Google Colab e Google Sheets concluída!


In [ ]:
# ==================================================================
# === INÍCIO DO BLOCO DE EXPORTAÇÃO ÚNICO PARA GOOGLE SHEETS ===
# (Este bloco substitui as seções EXPORTAÇÃO 1, 2 e 3)
# ==================================================================

# O 'import' foi movido para a Célula 1 (Autenticação)

print("\n--- INICIANDO EXPORTAÇÃO PARA O GOOGLE SHEETS ---")

try:
    # --- 1. CONFIGURAÇÃO DA PLANILHA ---
    NOME_DA_SUA_PLANILHA = "Dados | NPS"

    # Nomes das abas conforme solicitado
    NOME_ABA_DATA = "Data"
    NOME_ABA_BIGRAMS = "BiGrams"
    NOME_ABA_BIGRAMS_CLASS = "BiGrams_Class"

    # Abre a planilha (o 'gc' foi definido na Célula 1)
    sh = gc.open(NOME_DA_SUA_PLANILHA)
    print(f"Planilha '{NOME_DA_SUA_PLANILHA}' aberta com sucesso.")

    # ------------------------------------------------------------------
    # --- EXPORTAÇÃO "Data" (CORRIGIDA com 'Ano' e 'Centro') ---
    # ------------------------------------------------------------------
    print(f"\nIniciando exportação para a aba: '{NOME_ABA_DATA}'...")
    try:
        ws_data = sh.worksheet(NOME_ABA_DATA)
        print(f"Aba '{NOME_ABA_DATA}' encontrada.")
    except gspread.WorksheetNotFound:
        ws_data = sh.add_worksheet(title=NOME_ABA_DATA, rows=100, cols=20)
        print(f"Aba '{NOME_ABA_DATA}' não encontrada, criando...")

    # Define a lista de colunas exatas solicitadas pelo usuário
    colunas_exportar_usuario = [
        'Ano', 'Centro', 'Mes', 'Semana', 'Data Avaliação', 'Comentario',
        'classificacao', 'NPS', 'is_valid', 'is_generic',
        'sentimento_modelo', 'topico_modelo'
    ]

    # Verifica quais dessas colunas realmente existem no DataFrame df_proc
    # Assumindo que 'df_proc' é o DataFrame 'Data' final a ser exportado
    colunas_validas = [col for col in colunas_exportar_usuario if col in df_proc.columns]

    print(f"Colunas a serem exportadas para '{NOME_ABA_DATA}': {colunas_validas}")

    # ==================================================================
    # === INÍCIO DA CORREÇÃO DE FORMATO DA DATA ===
    # ==================================================================
    print("Formatando a coluna 'Data Avaliação' para YYYY-MM-DD...")
    if 'Data Avaliação' in colunas_validas:
        try:
            # 1. Garante que a coluna é um objeto datetime
            #    errors='coerce' transforma datas inválidas em NaT (Not a Time)
            df_proc['Data Avaliação'] = pd.to_datetime(df_proc['Data Avaliação'], errors='coerce')

            # 2. Formata a data para a string 'YYYY-MM-DD'
            df_proc['Data Avaliação'] = df_proc['Data Avaliação'].dt.strftime('%Y-%m-%d')

            # 3. Substitui 'NaT' (resultado do strftime em datas nulas/inválidas)
            #    por None para que a célula no Google Sheets fique vazia.
            df_proc['Data Avaliação'] = df_proc['Data Avaliação'].replace('NaT', None)

            print("Coluna 'Data Avaliação' formatada com sucesso.")

        except Exception as e:
            print(f"Aviso: Falha ao tentar formatar a coluna 'Data Avaliação'. Erro: {e}")
    else:
        print("Aviso: Coluna 'Data Avaliação' não encontrada para formatação.")
    # ==================================================================
    # === FIM DA CORREÇÃO DE FORMATO DA DATA ===
    # ==================================================================

    # Limpa e envia os dados
    ws_data.clear()

    # ----- ESTA É A FUNÇÃO CORRIGIDA -----
    # Exporta o DataFrame 'df_proc' APENAS com as colunas válidas solicitadas
    set_with_dataframe(ws_data, df_proc[colunas_validas], resize=True)
    print(f"✅ Dados de Classificação enviados para a aba '{NOME_ABA_DATA}'.")

    # ------------------------------------------------------------------
    # --- EXPORTAÇÃO "BiGrams" (CORRIGIDA para exportar 'Centro') ---
    # ------------------------------------------------------------------
    print(f"\nIniciando exportação para a aba: '{NOME_ABA_BIGRAMS}'...")
    if 'all_tfidf_results' in globals() and all_tfidf_results:
        try:
            ws_bigrams = sh.worksheet(NOME_ABA_BIGRAMS)
            print(f"Aba '{NOME_ABA_BIGRAMS}' encontrada.")
        except gspread.WorksheetNotFound:
            ws_bigrams = sh.add_worksheet(title=NOME_ABA_BIGRAMS, rows=100, cols=10)
            print(f"Aba '{NOME_ABA_BIGRAMS}' não encontrada, criando...")

        # Renomeia (sem mexer no 'Centro')
        df_para_exportar_tfidf = df_tfidf_completo.rename(columns={
            'term': 'Bigrama',
            'score': 'Score_TFIDF',
            'frequency': 'Frequencia'
        })

        # Define colunas (usando 'Centro' diretamente)
        colunas_exportar_tfidf = ['Centro', 'Bigrama', 'Classificacao', 'Score_TFIDF', 'Frequencia']
        colunas_exportar_tfidf = [col for col in colunas_exportar_tfidf if col in df_para_exportar_tfidf.columns] # Verificação

        # Limpa e envia os dados
        ws_bigrams.clear()

        # ----- ESTA É A FUNÇÃO CORRIGIDA -----
        set_with_dataframe(ws_bigrams, df_para_exportar_tfidf[colunas_exportar_tfidf], resize=True)
        print(f"✅ Dados de BiGrams (Antigo) enviados para a aba '{NOME_ABA_BIGRAMS}'.")
    else:
        print(f"⚠️ Pulando exportação para '{NOME_ABA_BIGRAMS}': Nenhum resultado de TF-IDF foi gerado.")


    # ------------------------------------------------------------------
    # --- EXPORTAÇÃO "BiGrams_Class" (Bigramas por Tópico) ---
    # ------------------------------------------------------------------
    print(f"\nIniciando exportação para a aba: '{NOME_ABA_BIGRAMS_CLASS}'...")
    if 'all_tfidf_results_topico' in globals() and all_tfidf_results_topico:
        try:
            ws_bigrams_class = sh.worksheet(NOME_ABA_BIGRAMS_CLASS)
            print(f"Aba '{NOME_ABA_BIGRAMS_CLASS}' encontrada.")
        except gspread.WorksheetNotFound:
            ws_bigrams_class = sh.add_worksheet(title=NOME_ABA_BIGRAMS_CLASS, rows=100, cols=10)
            print(f"Aba '{NOME_ABA_BIGRAMS_CLASS}' não encontrada, criando...")

        # Renomeia e define colunas
        df_para_exportar_topico = df_tfidf_topico_completo.rename(columns={
            'term': 'Bigrama', 'score': 'Score_TFIDF', 'frequency': 'Frequencia'
        })
        colunas_exportar_topico = ['Centro', 'Bigrama', 'Sentimento', 'Categoria', 'Score_TFIDF', 'Frequencia']
        colunas_exportar_topico = [col for col in colunas_exportar_topico if col in df_para_exportar_topico.columns] # Verificação

        # Limpa e envia os dados
        ws_bigrams_class.clear()

        # ----- ESTA É A FUNÇÃO CORRIGIDA -----
        set_with_dataframe(ws_bigrams_class, df_para_exportar_topico[colunas_exportar_topico], resize=True)
        print(f"✅ Dados de BiGrams por Classe enviados para a aba '{NOME_ABA_BIGRAMS_CLASS}'.")
    else:
        print(f"⚠️ Pulando exportação para '{NOME_ABA_BIGRAMS_CLASS}': Nenhum resultado de TF-IDF por Tópico/Sentimento foi gerado.")

    print(f"\n\n🎉 SUCESSO! Exportação para Google Sheets concluída!")
    print(f"Acesse sua planilha aqui: {sh.url}")

# --- 5. TRATAMENTO DE ERROS GERAIS ---
except gspread.exceptions.SpreadsheetNotFound:
    print(f"--- ERRO FATAL ---")
    print(f"Planilha '{NOME_DA_SUA_PLANILHA}' não encontrada no seu Google Drive.")
    print("Verifique se o nome está EXATAMENTE igual e se você deu permissão na célula de autenticação.")
except NameError as e:
    print(f"--- ERRO FATAL: VARIAVEL NÃO DEFINIDA ---")
    print(f"Ocorreu um erro: {e}. Isso pode acontecer se uma variavel (ex: 'df_proc' ou 'df_tfidf_completo') não foi criada.")
    print("Por favor, execute todas as células anteriores do notebook.")
except Exception as e:
    print(f"--- ERRO FATAL DURANTE A EXPORTAÇÃO ---")
    print(f"Ocorreu um erro inesperado: {e}")

print("\n--- SCRIPT COMPLETO CONCLUÍDO ---")


--- INICIANDO EXPORTAÇÃO PARA O GOOGLE SHEETS ---
Planilha 'Dados | NPS' aberta com sucesso.

Iniciando exportação para a aba: 'Data'...
Aba 'Data' encontrada.
Colunas a serem exportadas para 'Data': ['Ano', 'Centro', 'Mes', 'Semana', 'Data Avaliação', 'Comentario', 'classificacao', 'NPS', 'is_valid', 'is_generic', 'sentimento_modelo', 'topico_modelo']
Formatando a coluna 'Data Avaliação' para YYYY-MM-DD...
Coluna 'Data Avaliação' formatada com sucesso.
✅ Dados de Classificação enviados para a aba 'Data'.

Iniciando exportação para a aba: 'BiGrams'...
Aba 'BiGrams' encontrada.
✅ Dados de BiGrams (Antigo) enviados para a aba 'BiGrams'.

Iniciando exportação para a aba: 'BiGrams_Class'...
Aba 'BiGrams_Class' encontrada.
✅ Dados de BiGrams por Classe enviados para a aba 'BiGrams_Class'.


🎉 SUCESSO! Exportação para Google Sheets concluída!
Acesse sua planilha aqui: https://docs.google.com/spreadsheets/d/1n4Y5BTuvO6pcMKlCv2WsvpAVCCidD4SVou86i0EJEvk

--- SCRIPT COMPLETO CONCLUÍDO ---
